## We are going to implement GoogLeNet and train it on CIFAR-10.

# [Link to my Youtube Video Explaining this whole Notebook](https://www.youtube.com/watch?v=CZNYrkdDrmQ&list=PLxqBkZuBynVRyOJs4RWmB_fKlOVe5S8CR&index=12)

[![Imgur](https://imgur.com/a5lSG5y.png)](https://www.youtube.com/watch?v=CZNYrkdDrmQ&list=PLxqBkZuBynVRyOJs4RWmB_fKlOVe5S8CR&index=12)



In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from torchsummary import summary

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

from GoogLeNet import *
from train import *

In [11]:
model = GoogLeNet()

model.to(device)
summary(model, (3, 96, 96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]           9,472
       BatchNorm2d-2           [-1, 64, 48, 48]             128
              ReLU-3           [-1, 64, 48, 48]               0
         ConvBlock-4           [-1, 64, 48, 48]               0
         MaxPool2d-5           [-1, 64, 24, 24]               0
            Conv2d-6           [-1, 64, 24, 24]           4,160
       BatchNorm2d-7           [-1, 64, 24, 24]             128
              ReLU-8           [-1, 64, 24, 24]               0
         ConvBlock-9           [-1, 64, 24, 24]               0
           Conv2d-10          [-1, 192, 24, 24]         110,784
      BatchNorm2d-11          [-1, 192, 24, 24]             384
             ReLU-12          [-1, 192, 24, 24]               0
        ConvBlock-13          [-1, 192, 24, 24]               0
        MaxPool2d-14          [-1, 192,

## Loading CIFAR-10

In [12]:
def cifar_dataloader():
    
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.5], std=[0.5])])
            
    # Input Data in Local Machine
    # train_dataset = datasets.CIFAR10('../input_data', train=True, download=True, transform=transform)
    # test_dataset = datasets.CIFAR10('../input_data', train=False, download=True, transform=transform)
    
    # Input Data in Google Drive
    train_dataset = datasets.CIFAR10('/content/drive/MyDrive/All_Datasets/CIFAR10', train=True, download=True, transform=transform)
    
    test_dataset = datasets.CIFAR10('/content/drive/MyDrive/All_Datasets/CIFAR10', train=False, download=True, transform=transform)

    # Split dataset into training set and validation set.
    train_dataset, val_dataset = random_split(train_dataset, (45000, 5000))
    
    print("Image shape of a random sample image : {}".format(train_dataset[0][0].numpy().shape), end = '\n\n')
    
    print("Training Set:   {} images".format(len(train_dataset)))
    print("Validation Set:   {} images".format(len(val_dataset)))
    print("Test Set:       {} images".format(len(test_dataset)))
    
    BATCH_SIZE = 128

    # Generate dataloader
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    return train_loader, val_loader, test_loader

In [13]:
train_loader, val_loader, test_loader = cifar_dataloader()

Files already downloaded and verified
Files already downloaded and verified
Image shape of a random sample image : (3, 32, 32)

Training Set:   45000 images
Validation Set:   5000 images
Test Set:       10000 images


## c) Training the model

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
train_epoch_loss_history, val_epoch_loss_history = train_model(model, train_loader, val_loader, criterion, optimizer)

[For Epoch 1/15]: train-loss = 2.40094 | train-acc = 0.453 | val-loss = 2.10815 | val-acc = 0.524
[For Epoch 2/15]: train-loss = 1.72943 | train-acc = 0.624 | val-loss = 1.70226 | val-acc = 0.622
[For Epoch 3/15]: train-loss = 1.40953 | train-acc = 0.699 | val-loss = 1.62197 | val-acc = 0.652
[For Epoch 4/15]: train-loss = 1.18186 | train-acc = 0.750 | val-loss = 1.30169 | val-acc = 0.720
[For Epoch 5/15]: train-loss = 1.00871 | train-acc = 0.789 | val-loss = 1.36148 | val-acc = 0.725
[For Epoch 6/15]: train-loss = 0.87731 | train-acc = 0.817 | val-loss = 1.20267 | val-acc = 0.751
[For Epoch 7/15]: train-loss = 0.75602 | train-acc = 0.840 | val-loss = 1.40170 | val-acc = 0.735
[For Epoch 8/15]: train-loss = 0.66466 | train-acc = 0.859 | val-loss = 1.26839 | val-acc = 0.763
[For Epoch 9/15]: train-loss = 0.56721 | train-acc = 0.881 | val-loss = 1.31561 | val-acc = 0.757
[For Epoch 10/15]: train-loss = 0.49533 | train-acc = 0.898 | val-loss = 1.26927 | val-acc = 0.771
[For Epoch 11/15]: 

## Evaluating model

In [16]:
model = GoogLeNet()
model.load_state_dict(torch.load('/content/sample_data/googlenet_model'))

<All keys matched successfully>

In [17]:
num_test_samples = 10000
correct = 0 

model.eval().cuda()

with  torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # Make predictions.
        prediction, _, _ = model(inputs)

        # Retrieve predictions indexes.
        _, predicted_class = torch.max(prediction.data, 1)

        # Compute number of correct predictions.
        correct += (predicted_class == labels).float().sum().item()

test_accuracy = correct / num_test_samples

print('Test accuracy: {}'.format(test_accuracy))

Test accuracy: 0.7571


## Bonus Sections - GoogLeNet Basic Explanations as talked about the associated YouTube Video 



### Original Problem

Salient parts in the image can have extremely large variation in size. For instance, an image with a cat can be either of the following, as shown below. The area occupied by the cat is different in each image.

![Imgur](https://imgur.com/uAWiQaC.png)

- Because of this huge variation in the location of the information, choosing the right kernel size for the convolution operation becomes tough. A larger kernel is preferred for information that is distributed more globally, and a smaller kernel is preferred for information that is distributed more locally.

- Very deep networks are prone to overfitting. It also hard to pass gradient updates through the entire network.

- Naively stacking large convolution operations is computationally expensive.

The Solution:

Why not have filters with multiple sizes operate on the same level? The network essentially would get a bit “wider” rather than “deeper”. The authors designed the inception module to reflect the same.

The below image is the “naive” inception module. It performs convolution on an input, with 3 different sizes of filters (1x1, 3x3, 5x5). Additionally, max pooling is also performed. The outputs are concatenated and sent to the next inception module.

![Imgur](https://imgur.com/x5pl2QB.png)

However,  deep neural networks are computationally expensive. To make it cheaper, the authors limit the number of input channels by adding an extra 1x1 convolution before the 3x3 and 5x5 convolutions. Though adding an extra operation may seem counterintuitive, 1x1 convolutions are far more cheaper than 5x5 convolutions, and the reduced number of input channels also help. Do note that however, the 1x1 convolution is introduced after the max pooling layer, rather than before.


![Imgur](https://imgur.com/zqrJ2jo.png)

Using the dimension reduced inception module, a neural network architecture was built. This was popularly known as GoogLeNet (Inception v1). The architecture is shown below:


Proposed Architectural Details
The paper proposes a new type of architecture – GoogLeNet or Inception v1. It is basically a convolutional neural network (CNN) which is 27 layers deep. Below is the model summary:


![Imgur](https://imgur.com/8Br0HLk.png)

Notice in the above image that there is a layer called inception layer. This is actually the main idea behind the paper’s approach.

![Imgur](https://imgur.com/B281dyr.png)


### (Inception Layer) is a combination of all those layers (namely, 1×1 Convolutional layer, 3×3 Convolutional layer, 5×5 Convolutional layer) with their output filter banks concatenated into a single output vector forming the input of the next stage.


===========================================================================

### Inception Module:

The inception module is different from previous architectures such as AlexNet, ZF-Net. In this architecture, there is a fixed convolution size for each layer.

In theory you can have as many filter sizes as possible, but the Inception Architecture is restricted to filter sizes 1 × 1, 3 × 3 and 5 × 5. The small filters help capture the local details and features whereas spread out features of higher abstraction are captured by the larger filters. A 3 × 3 max pooling is also added to the Inception architecture, because, why not? Historically, it has been found that pooling layers make the network work better, so might as well add them!

#### In the Inception module 1×1, 3×3, 5×5 convolution and 3×3 max pooling performed in a parallel way at the input and the output of these are stacked together to generate final output. The idea behind is - that the convolution filters of different sizes will handle objects at multiple scale better.


===========================================================================